In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import missingno as msno

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_validate

from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')


from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

%matplotlib inline  

housing_raw = pd.read_csv('Ames_HousePrice.csv', index_col=0)
price = housing_raw['SalePrice'].copy()
housing_raw.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [2]:
df_num = housing_raw[['GrLivArea', 'OverallQual', 'TotalBsmtSF', '1stFlrSF','GarageCars', 'GarageArea', 'YearBuilt', 'FullBath', 'YearRemodAdd', 'MasVnrArea', 'TotRmsAbvGrd', 'Fireplaces', 'BsmtFinSF1','LotFrontage','WoodDeckSF']].copy()
df_num.loc[df_num['GarageCars'].isnull(), 'GarageCars'] = 2
# df_num.loc[df_num['GarageArea'].isnull(), 'GarageArea'] = 416
df_num.loc[df_num['LotFrontage'].isnull(), 'LotFrontage'] = df_num['LotFrontage'].mean()
# df_num['LotFrontage'].fillna(df_num['LotFrontage'].mean(), inplace = True)

# Because there is only 1 null value for area, we just fill in with the Year the house was built
# df_num.loc[df_num['GarageArea'].isnull(), 'GarageYrBlt'] = df_num.loc[df_num['GarageArea'].isnull(),'YearBuilt']
df_num['GarageArea'].fillna(housing_raw[housing_raw['GarageType']== 'Detchd']['GarageArea'].mean(), inplace = True)
df_num.fillna(0,inplace = True)

In [4]:
df_cat = housing_raw.select_dtypes(['object']).copy()
df_cat.fillna('None', inplace = True)

# Define the mapping
po_ex_ordinal_mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['ExterQual'] = df_cat['ExterQual'].map(po_ex_ordinal_mapping)
df_cat['ExterCond'] = df_cat['ExterCond'].map(po_ex_ordinal_mapping)
df_cat['KitchenQual'] = df_cat['KitchenQual'].map(po_ex_ordinal_mapping)
df_cat['HeatingQC'] = df_cat['HeatingQC'].map(po_ex_ordinal_mapping)


# Defind the mapping
none_ex6_ordinal_mapping = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['BsmtQual'] = df_cat['BsmtQual'].map(none_ex6_ordinal_mapping)
df_cat['BsmtCond'] = df_cat['BsmtCond'].map(none_ex6_ordinal_mapping)
df_cat['FireplaceQu'] = df_cat['FireplaceQu'].map(none_ex6_ordinal_mapping)
df_cat['GarageQual'] = df_cat['GarageQual'].map(none_ex6_ordinal_mapping)
df_cat['GarageCond'] = df_cat['GarageCond'].map(none_ex6_ordinal_mapping)
 

# Define the mapping
none_fin_ordinal_mapping = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin':3}
# Applying the mapping
df_cat['GarageFinish'] = df_cat['GarageFinish'].map(none_fin_ordinal_mapping)


# Define the mapping
none_ex5_ordinal_mapping = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd':3, 'Ex':4}
# Applying the mapping
df_cat['PoolQC'] = df_cat['PoolQC'].map(none_ex5_ordinal_mapping)

# Define mapping
binary_mapping = {'Y': 1, 'N':0}
df_cat['CentralAir'] = df_cat['CentralAir'].map(binary_mapping)


df_cat = df_cat[['BldgType','HouseStyle','RoofStyle','MasVnrType','SaleCondition','GarageQual']].copy()
df_cat

# df_cat_dummy = pd.get_dummies(df_cat, drop_first=True)
# df_cat_dummy

,BldgType,HouseStyle,RoofStyle,MasVnrType,SaleCondition,GarageQual
1,1Fam,1Story,Gable,None,Normal,3
2,TwnhsE,1Story,Gable,BrkFace,Normal,3
3,1Fam,1Story,Hip,None,Normal,3
4,1Fam,2Story,Gable,None,Normal,3
5,1Fam,2Story,Gable,None,Normal,3
...,...,...,...,...,...,...
759,1Fam,1.5Unf,Gable,None,Normal,2
760,1Fam,1Story,Hip,None,Normal,3
761,Duplex,2Story,Gable,None,Normal,3
762,1Fam,2Story,Gable,BrkFace,Normal,3


In [ ]:
df = pd.concat([df_num, df_cat_dummy], axis = 1)
df

In [5]:
x = df
y = price

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=42)

In [25]:
rf = RandomForestRegressor()
param_grid = {
    'n_estimators': [140, 141, 150],
    'min_samples_split':[12, 15, 20],
    'max_depth': [30, 35, 40]

}
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [30, 35, 40],
                         'min_samples_split': [12, 15, 20],
                         'n_estimators': [140, 141, 150]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
best_forest = grid_search.best_estimator_
print(best_forest)
best_forest.score(x_test, y_test)

RandomForestRegressor(max_depth=30, min_samples_split=15, n_estimators=141)


0.8814859546459342